## Import Datasets

**Make sure that you've downloaded the required human and dog datasets:
Download the dog dataset. Unzip the folder and place it in this project's home directory, at the location /dog_images.
Download the human dataset. Unzip the folder and place it in the home directory, at location /lfw.**

In [ ]:
!wget https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip
!wget https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/lfw.zip

In [ ]:
#!unzip dogImages.zip
#!unzip lfw.zip

In [ ]:
from glob import glob

# load filenames for human and dog images
human_files = np.array(glob("./lfw/*/*"))
dog_files = np.array(glob("./dogImages/*/*/*"))

# print number of images in each dataset
print('There are %d total human images.' % len(human_files))
print('There are %d total dog images.' % len(dog_files))

In [ ]:
from PIL import Image
image = Image.open(dog_files[1])
plt.imshow(image)

## Obtain Pre-trained VGG-16 Model
**The code cell below downloads the VGG-16 model, along with weights that have been trained on ImageNet, a very large, very popular dataset used for image classification and other vision tasks. ImageNet contains over 10 million URLs, each linking to an image containing an object from one of 1000 categories.**

In [ ]:
from keras.applications.vgg16 import VGG16
model = VGG16()

In [ ]:
model.summary()

First, we can use the load_img() function to load the image and resize it to the required size of 224×224 pixels.

In [ ]:
from keras.preprocessing.image import load_img

image = load_img('./dogImages/train/001.Affenpinscher/Affenpinscher_00002.jpg',target_size=(224, 224))

Next, we can convert the pixels to a NumPy array so that we can work with it in Keras. We can use the img_to_array() function for this. We only have one sample (one image). We can reshape the array by calling reshape() and adding the extra dimension.

In [ ]:
from keras.preprocessing.image import img_to_array
# convert the image pixels to a numpy array
image = img_to_array(image)
# reshape data for the model
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))

Keras provides a function called preprocess_input() to prepare new input for the network.



In [ ]:
from keras.applications.vgg16 import preprocess_input
# prepare the image for the VGG model
image = preprocess_input(image)

We are now ready to make a prediction for our loaded and prepared image.

